# C3 Project 1: Image Classification

In [5]:
from main import test, train, TrainDataset, TestDataset
from bovw import BOVW
# metrics 
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, roc_auc_score, roc_curve, ConfusionMatrixDisplay,
)
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

## Charge the data

In [6]:
data_train = TrainDataset()
data_test = TestDataset()

### Experiment 1

In [7]:
# HYPERPARAMETERS
detector_type = "SIFT"
codebook_size = 50
classifier_algorithm = "LogisticRegression"
cache_train = "./cache_train_descriptor.pkl"
cache_test = "./cache_test_descriptor.pkl"

## Run experiment

In [8]:
bovw = BOVW(detector_type=detector_type, codebook_size=codebook_size)

if classifier_algorithm == 'LogisticRegression':
        classifier = LogisticRegression(class_weight="balanced")
elif classifier_algorithm == 'SVM':
        classifier = SVC(class_weight="balanced", probability=True)

# TRAIN
y_pred_train, y_probas_train, labels_train = train(dataset=data_train, bovw=bovw, classifier=classifier, cache_file=cache_train)
train_acc = accuracy_score(y_true=labels_train, y_pred=y_pred_train)
print("Accuracy on Phase[Train]:", train_acc)

# TEST
y_pred_test, y_probas_test, labels_test = test(dataset=data_test, bovw=bovw, classifier=classifier, cache_file=cache_test)
test_acc = accuracy_score(y_true=labels_test, y_pred=y_pred_test)
print("Accuracy on Phase[Test]:", test_acc)

Phase[Train]: Loading descriptors from ./cache_train_descriptor.pkl
Fitting the codebook (took 1.44 seconds)
Computing the bovw histograms
Fitting the classifier
Accuracy on Phase[Train]: 0.2751207174063003
Phase[Test]: Loading descriptors from ./cache_test_descriptor.pkl
Computing the bovw histograms
Predicting the values
Accuracy on Phase[Test]: 0.2790909090909091
